In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-06"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
18065,2024-11-06,Japão B.League,07:05,Chiba,Gunma,1.41,2.62,154.5,1.83,1.83,-6.5,1.99,1.70,S87c6Fpn,0.709220,0.381679,0.546448,0.546448,0.090899,110.224,17.745778,0.160997,2.4,1.341641,0.559017,83.85,1.396,0.139392,0.099851,41.0,130.324,58.662960,0.450132,2.4,1.341641,0.559017,114.24,1.640,0.973679,0.593706,69.0,65,102,1.29,1.12,118.966,186.212,0.424615,0.000000,0.111144,-1.52,-0.304,-1.348684,0.633929,0.7,0.066071,-2.37,-0.474,-3.417722,0.405954,0.3,-0.105954
18066,2024-11-06,Japão B.League,07:05,Hiroshima D.,Osaka,2.30,1.52,161.5,1.86,1.80,3.5,1.92,1.75,fgxs4pLF,0.434783,0.657895,0.537634,0.555556,0.092677,319.416,141.990272,0.444531,0.6,1.341641,2.236068,394.20,4.114,1.760676,0.427972,-48.0,164.110,54.818646,0.334036,1.2,1.643168,1.369306,244.44,1.968,0.718171,0.364924,-12.0,73,84,5.40,2.91,133.110,175.846,0.288766,0.023184,0.065509,-2.42,-0.484,-2.685950,0.694512,0.6,-0.094512,-2.55,-0.510,-1.019608,0.417722,0.2,-0.217722
18067,2024-11-06,Japão B.League,07:05,Hokkaido,Sendai,1.56,2.21,148.5,1.83,1.83,-4.5,1.99,1.70,YonOiGVh,0.641026,0.452489,0.546448,0.546448,0.093514,218.496,78.986620,0.361501,1.2,1.643168,1.369306,173.25,2.916,0.825821,0.283203,-20.0,129.038,42.630148,0.330369,1.8,1.643168,0.912871,100.44,1.796,0.403522,0.224678,-11.0,75,54,2.31,1.86,300.070,223.252,0.243830,0.000000,0.111144,-2.69,-0.538,-1.040892,0.293313,0.4,0.106687,-0.14,-0.028,-43.214286,0.372559,0.4,0.027441
18068,2024-11-06,Japão B.League,07:05,Ibaraki Robots,Utsunomiya Brex,5.91,1.08,156.5,1.80,1.86,12.5,2.00,1.68,GUwulYUH,0.169205,0.925926,0.555556,0.537634,0.095131,379.182,251.007622,0.661971,0.0,0.000000,NaN,246.01,4.854,2.858335,0.588862,-79.0,107.948,19.625094,0.181801,3.0,0.000000,0.000000,137.78,1.400,0.216217,0.154441,37.0,73,83,3.37,1.66,366.866,98.104,0.977203,0.023184,0.122975,7.61,1.522,3.226018,0.260751,0.3,0.039249,1.00,0.200,0.400000,0.824880,0.9,0.075120
18069,2024-11-06,Japão B.League,07:05,Koshigaya Alphas,Nagasaki,3.07,1.30,150.5,1.83,1.83,6.5,1.97,1.71,lrxmnCaU,0.325733,0.769231,0.546448,0.546448,0.094964,218.036,47.082060,0.215937,0.0,0.000000,NaN,227.74,3.224,0.724037,0.224577,-64.0,152.080,39.559924,0.260126,1.2,1.643168,1.369306,133.20,2.084,0.408571,0.196051,-24.0,59,74,3.86,1.80,305.890,148.526,0.572805,0.000000,0.099917,-1.14,-0.228,-9.078947,0.000000,0.0,0.000000,1.96,0.392,0.765306,0.549017,0.7,0.150983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18182,2024-11-06,Eua Ncaa,21:00,High Point,Coppin State,1.01,16.00,143.5,1.83,1.83,-22.5,1.83,1.83,CvS8eIEk,0.990099,0.062500,0.546448,0.546448,0.052599,102.818,17.622702,0.171397,1.8,1.643168,0.912871,105.86,1.282,0.174557,0.136160,30.0,355.620,225.286679,0.633504,0.0,0.000000,NaN,418.20,5.906,3.257933,0.551631,-79.0,79,51,1.34,8.20,106.218,377.390,1.246271,0.000000,0.000000,-1.35,-0.270,-0.037037,0.826448,0.8,-0.026448,-5.00,-1.000,-15.000000,0.158497,0.0,-0.158497
18183,2024-11-06,Eua Ncaa,00:00,Washington,UC Davis,1.07,9.80,146.5,1.80,1.95,-16.5,2.00,1.71,YqJULBfi,0.934579,0.102041,0.555556,0.512821,0.036620,265.360,223.094064,0.840722,1.2,1.643168,1.369306,16

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
18078,13:30,Finlândia Korisliiga,UU-Korihait,Kouvot,1.70,Back Home
18100,23:00,Eua Ncaa,Pepperdine,Western Illinois,1.61,Back Home
18108,23:15,México Lnbp,Fuerza Regia,Halcones de Xalapa,1.57,Back Home
18172,14:00,Europa Copa Europeia Da Fiba,Anwil Wloclawek,Sassari,1.57,Back Home
18175,14:30,Europa Liga Dos Campeões,Peristeri,Manisa,1.56,Back Home
